# Dynamics of the system

In [47]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, vlatex
import numpy as np
from IPython.display import display, Math
from phaseportrait import PhasePortrait2D

# The equations

<img src="assets/system_image.jpg" alt="alt text" width="600"/>

## Variables & symbolic constants  

Constants:
* $m_d, m_B, m_E, m_{BE}, m_{AD}$ -- masses of disk, connector, end-effector and rods $BE$ and $AD$ respectively
* $R, l_{BE}, l_{AD}$ -- radius of the disk, length of $BE$ and $AD$ respectively
* $I_e$ -- inertia of end-effector around center of mass
* Coordinates of point $A$
* Angle of rotation of gravity force $\theta_g$ 

In [48]:
m_d, m_B, m_E, m_BE, m_AD = sp.symbols('m_d, m_B, m_E, m_{BE}, m_{AD}')
m_rod = m_B + m_E + m_BE + m_AD
R, l_BE, l_AD = sp.symbols('R, l_{BE}, l_{AD}')
I_e = sp.symbols('I_e')
x_A, y_A = sp.symbols('x_A, y_A')
r_A = sp.Matrix([x_A, y_A])
theta_g = sp.symbols('theta_g')
g = sp.symbols('g')

I_d = m_d * R**2 / 2

Variables:
* $\theta(t)$ -- generalized coordinate, angle of rotations of the disk
* $\tau_{ext}$ -- external torque, applied to the disk

In [49]:
t = sp.symbols('t')
theta, tau_ext = dynamicsymbols('theta, tau_{ext}') 
dtheta = theta.diff(t)

## System's points as function of coordinate

There are many points & parameters in the system, which is important to derive from the system coordinates. These are:
* $r_B, r_M, r_F, r_E$ -- coordinates of points $B, M, F, E$ respectively
* Angle $\alpha$ between rod and horizon
* $l_{com}$ is a specific parameter, representing distance between point B and the center of mass of the inner rod
* $\frac{r_B - r_a}{||r_B - r_A||}$, represented as `r_AB_norm` is a normal vector, collinear with the rod

In [50]:
x_B, y_B = R * sp.cos(theta), R * sp.sin(theta)
r_B = sp.Matrix([x_B, y_B])

alpha = sp.atan2(y_B - y_A, x_B - x_A)
r_AB_norm = sp.Matrix([sp.cos(alpha), sp.sin(alpha)])

r_M = r_A + r_AB_norm * l_AD / 2
r_F = r_B + r_AB_norm * l_BE / 2
r_E = r_B + r_AB_norm * l_BE

l_com = (r_B - r_A).norm() - l_AD / 2

It is also quite handy to have an angular velocity $\dot{\alpha}$ to calculate kinetic energy of the system. Moreover, the variable $K$ is introduced, such that:
$$\dot{\alpha} = K_\alpha \dot{\theta}$$

In [51]:
dalpha = sp.diff(alpha, t)
K_alpha = sp.diff(alpha, theta)

display(Math(f'$\dot{{\\alpha}} = {vlatex(dalpha)} = K \dot{{\\theta}}$'))

<IPython.core.display.Math object>

## Energy of the system

### Velocity of rod's center of mass

One of the key component of the kinetic energy of the system is the velocity of center of mass. 

The position itself could be expressed as follows:
$$r_{com} = \frac{1}{m_{rod}} (m_{AD} r_M + m_B r_B + m_{BE} r_F + m_E r_E)$$

Despite the `sympy` is cabable of differentiating the velocity symbolically, it is useful to derive these equations manually:
$$v_{com} = \frac{1}{m_{rod}}\left( (m_{AD} \frac{l_{AD}}{2} + m_{BE} \frac{l_{BE}}{2} + m_E l_{BE}) \begin{bmatrix} -\sin \alpha \\ \cos \alpha \end{bmatrix} K_\alpha +  (m_B + m_{BE} + m_E) \begin{bmatrix} -\sin \theta \\ \cos \theta \end{bmatrix}\right) \dot{\theta} = D \dot{\theta}$$

In [52]:
r_com = (m_AD * r_M + m_B * r_B + m_BE * r_F + m_E * r_E) / m_rod
D = ((m_AD * l_AD / 2 + m_BE * l_BE / 2 + m_E * l_BE) * sp.Matrix([-sp.sin(alpha), sp.cos(alpha)]) * K_alpha + (m_B + m_BE + m_E) * r_AB_norm.diff(theta)) / m_rod

### Inertia of the rod

The intertia of the rod around rotational point $B$ is as follows:
$$I_r = \frac{m_{AD}}{3} (\frac{l_{AD}^2}{4} + 3 l_{com}^2) + \frac{1}{3} m_{BE} l_{BE}^2 + I_E + m_E l_{BE}^2$$

In [53]:
I_r = m_AD / 3 * (l_AD ** 2 / 4 + 3 * l_com**2) + m_BE * l_BE**2 / 3 + I_e + m_E * l_BE ** 2

### Kinetic Energy

Using all calculations above, the kinetic energy could be formulated as follows:
$$K = \frac{1}{2} m_{rod} v_{com}^T v_{com} + \frac{1}{2} I_d \dot{\theta}^2 + \frac{1}{2} I_r \dot{\alpha}^2 = \frac{1}{2} M \dot{\theta}^2,$$
where:
$$M = m_{rod} D^T D  + I_d + I_r K^2_\alpha$$

In [54]:
M = m_rod * D.norm()**2 + I_d + I_r * K_alpha**2
M

R**2*m_d/2 + (-R*(-R*sin(theta(t)) + y_A)*sin(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) + R*(R*cos(theta(t)) - x_A)*cos(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))**2*(I_e + l_{BE}**2*m_E + l_{BE}**2*m_{BE}/3 + m_{AD}*(l_{AD}**2/4 + 3*(-l_{AD}/2 + sqrt(Abs(R*sin(theta(t)) - y_A)**2 + Abs(R*cos(theta(t)) - x_A)**2))**2)/3) + (Abs((-(R*sin(theta(t)) - y_A)*(R*(-R*sin(theta(t)) + y_A)*sin(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) - R*(R*cos(theta(t)) - x_A)*cos(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))*(l_{AD}*m_{AD}/2 + l_{BE}*m_E + l_{BE}*m_{BE}/2)/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) + (R*sin(theta(t))/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) - (R*cos(theta(t)) - x_A)*(-R*(R*sin(theta(t)) - y_A)*cos(theta(t)) + R*(R*cos(theta(t)) - x_A)*sin(theta(t)))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)**(3/2))*(m_B + m_E + m_{BE}))/

### Potential energy

The only source of potential energy in the system is a gravitational force, affecting . Hence, the equation for a potential energy is:
$$P = m_{rod} g y_{com} \cos \theta_g$$

In [55]:
P = m_rod * g * r_com[1] * sp.cos(theta_g)
P

g*(R*m_B*sin(theta(t)) + m_E*(R*sin(theta(t)) + l_{BE}*(R*sin(theta(t)) - y_A)/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)) + m_{AD}*(l_{AD}*(R*sin(theta(t)) - y_A)/(2*sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)) + y_A) + m_{BE}*(R*sin(theta(t)) + l_{BE}*(R*sin(theta(t)) - y_A)/(2*sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))))*cos(theta_g)

## Resulting equation

Given values of $M$, $P$, and external torque law $\tau_{ext}$, one might formulate an equation:
$$M \ddot{\theta} + \dot{M} \dot{\theta} - \frac{1}{2} \frac{d}{d\theta} (M\dot{\theta}^2) + \frac{dP}{d\theta} = \tau_{ext}$$ 

In [56]:
M_dot = sp.diff(M, t)
dMtheta_dtheta = sp.diff(M*dtheta**2, theta)
dP_dtheta = sp.diff(P, theta)

KeyboardInterrupt: 

In [ ]:
ddtheta = (-M_dot * dtheta + dMtheta_dtheta / 2 + dP_dtheta - tau_ext) / M

# Numerical computing

Unfortunately, due to high complexity of the resulting dynamics, `sp.lambdify()` fails to treat them in reasonable time. Hence, the function for $\ddot{\theta}$ has to be written manually.

## Constant initialization

In [ ]:
m_d, m_B, m_E, m_BE, m_AD = 0.1, 0.4, 0.6, 0.08, 0.05
m_rod = m_B + m_E + m_BE + m_AD
R, l_BE, l_AD = .12, .2, .3
I_e = 0.5
x_A, y_A = -0.09, -0.11
r_A = sp.Matrix([x_A, y_A])
theta_g = 0
g = 9.81

I_d = m_d * R**2 / 2

In [80]:
class ModelDynamics():
    def __init__(
            self, 
            m_d=.1, 
            m_B=.4, 
            m_E=.6, 
            m_BE=.08, 
            m_AD=.05, 
            R=.12, 
            l_BE=.2, 
            l_AD=.3, 
            I_e=.5, 
            x_A=-0.09, 
            y_A=-0.11, 
            theta_g=0
        ):
        self.m_d, self.m_B, self.m_E, self.m_BE, self.m_AD = m_d, m_B, m_E, m_BE, m_AD
        self.R, self.l_BE, self.l_AD = R, l_BE, l_AD
        self.I_e = I_e
        self.r_A = np.array([x_A, y_A])
        self.theta_g = theta_g
        self.g = 9.81

        self.no_input_error_text = "No input data is provided"

        self.t = None
        self.theta = None
        self.theta_dot = None
        self.tau = None

        self.update_constant_terms()
        self.reset_internal_params()


    def reset_internal_params(self):
        self.r_B = None
        self.r_B_dot = None
        self.alpha = None
        self.rod_norm = None
        self.r_M = None
        self.l_com = None
        self.l_com_dot = None
        self.r_F = None
        self.r_E = None
        
        self.K_alpha = None
        self.K_alpha_dot = None
        self.D = None
        self.D_dot = None
        self.dD_dtheta = None
        self.I_r = None
        self.I_r_dot = None
        self.M = None
        self.M_dot = None
        self.dM_dtheta = None
        self.P = None
        self.energy = None


    def update_constant_terms(self):
        self.m_rod = self.m_B + self.m_E + self.m_BE + self.m_AD
        
        self.I_d = self.m_d * self.R**2 / 2
        
        self.D_norm_component = (self.m_AD * self.l_AD / 2 + self.m_BE * self.l_BE / 2 + self.m_E * self.l_BE) / self.m_rod
        self.D_theta_component = (self.m_B + self.m_BE + self.m_E) / self.m_rod

    
    def set_parameters(self, theta, theta_dot, tau=0):
        self.theta = theta
        self.theta_dot = theta_dot
        self.tau = tau

        self.reset_internal_params()
    

    def _eval_r_B(self):
        if self.theta is None:
            raise RuntimeError(self.no_input_error_text)

        x_B = self.R * np.cos(self.theta)
        y_B = self.R * np.sin(self.theta)

        self.r_B = np.array([x_B, y_B])


    def _eval_r_B_dot(self):
        if self.theta is None or self.theta_dot is None:
            raise RuntimeError(self.no_input_error_text)
        x_B_dot = -self.R * np.sin(self.theta) * self.theta_dot
        y_B_dot = self.R * np.cos(self.theta) * self.theta_dot

        self.r_B_dot = np.array([x_B_dot, y_B_dot])


    def _eval_alpha(self):
        if self.r_B is None:
            self._eval_r_B()

        self.alpha = np.arctan2(self.r_B[1] - self.r_A[1], self.r_B[0] - self.r_A[0])


    def _eval_rod_norm(self):
        if self.alpha is None:
            self._eval_alpha()
        self.rod_norm = np.array([np.cos(self.alpha), np.sin(self.alpha)])

    
    def _eval_r_M(self):
        if self.alpha is None:
            self._eval_alpha()
        if self.rod_norm is None:
            self._eval_rod_norm()
        
        self.r_M = r_A + self.rod_norm * self.l_AD / 2

    
    def _eval_l_com(self):
        if self.r_B is None:
            self._eval_r_B()
        
        self.l_com = np.linalg.norm(self.r_B - self.r_A) - self.l_AD / 2
    

    def _eval_l_com_dot(self):
        if self.r_B is None:
            self._eval_r_B()
        if self.r_B_dot is None:
            self._eval_r_B_dot()
        
        self.l_com = (self.r_B[0] - self.r_A[0] + self.r_B[1] - self.r_A[1]) * (self.r_B_dot[0] + self.r_B_dot[1]) / np.linalg.norm(self.r_B - self.r_A)
    

    def _eval_r_F(self):
        if self.r_B is None:
            self._eval_r_B()
        if self.rod_norm is None:
            self._eval_rod_norm()
        
        self.r_F = self.r_B + self.rod_norm * self.l_BE/2
    

    def _eval_r_E(self):
        if self.r_B is None:
            self._eval_r_B()
        if self.rod_norm is None:
            self._eval_rod_norm()
        
        self.r_E = self.r_B + self.rod_norm * self.l_BE
    

    def _eval_K_alpha(self):
        if self.theta is None:
            raise RuntimeError(self.no_input_error_text)
        if self.r_B is None:
            self._eval_r_B

        Rcos = R * np.cos(self.theta)
        Rsin = R * np.sin(self.theta)

        denom = np.linalg.norm(self.r_B - self.r_A)

        self.K_alpha = (self.R**2 - self.r_B[1] - self.r_B[0]) / denom


    def _eval_K_alpha_dot(self):
        if self.theta is None or self.theta_dot is None:
            raise RuntimeError(self.no_input_error_text)
        if self.r_B is None:
            self._eval_r_B()

        sin_theta = np.sin(self.theta)
        cos_theta = np.cos(self.theta)

        denom = np.linalg.norm(self.r_B - self.r_A)**2
        
        self.K_alpha_dot = R * (-R**2 * self.x_A * sin_theta + \
            R**2 * self.y_A * cos_theta + \
                self.x_A**3 * sin_theta - \
                    self.x_A**2 * self.y_A * cos_theta + \
                        self.x_A * self.y_A**2 * sin_theta - \
                            self.y_A**3 * cos_theta) / denom * self.theta_dot


    def _eval_D(self):
        if self.theta is None:
            raise RuntimeError(self.no_input_error_text)
        if self.K_alpha is None:
            self._eval_K_alpha()

        self.D = self.D_norm_component * np.array([-np.sin(self.alpha), np.cos(self.theta)]) * K_alpha + \
            self.D_theta_component * np.array([-np.sin(self.theta), np.cos(self.theta)]) 
    

    def _eval_D_dot(self):
        if self.theta is None or self.theta_dot is None:
            raise RuntimeError(self.no_input_error_text)
        if self.K_alpha is None:
            self._eval_K_alpha()
        if self.K_alpha_dot is None:
            self._eval_K_alpha_dot()

        self.D_dot = self.D_norm_component * np.array([-np.sin(alpha), np.cos(alpha)]) * self.K_alpha_dot + \
            self.D_norm_component * np.array([-np.cos(alpha), -np.sin(alpha)]) * self.theta_dot * self.K_alpha + \
            self.D_theta_component * np.array([-np.cos(theta), -np.sin(theta)]) * self.theta_dot


    def _eval_I_r(self):
        if self.l_com is None:
            self._eval_l_com()
        
        self.I_r = self.m_AD * (self.l_AD ** 2 / 4 + 3 * self.l_com**2) + \
            self.m_BE * self.l_BE ** 2 / 3 + self.I_e + self.m_E * l_BE**2


    def _eval_I_r_dot(self):
        if self.l_com is None:
            self._eval_l_com()
        
        if self.l_com_dot is None:
            self._eval_l_com_dot()
        
        self.I_r = 6 * self.m_AD * self.l_com*self.l_com_dot


    def _eval_M(self):
        if self.D is None:
            self._eval_D()
        if self.I_r is None:
            self._eval_I_r()
        if self.K_alpha is None:
            self._eval_K_alpha()

        self.M = self.m_rod * self.D.T @ D + self.I_d + self.I_r * self.K_alpha ** 2


    def _eval_M_dot(self):
        if self.D is None:
            self._eval_D()
        if self.D_dot is None:
            self._eval_D_dot()
        if self.I_r is None:
            self._eval_I_r()
        if self.I_r_dot is None:
            self._eval_I_r_dot()
        if self.K_alpha is None:
            self._eval_K_alpha()
        if self.K_alpha_dot is None:
            self._eval_K_alpha_dot()

        self.M_dot =  2 * self.m_dot * self.D.T @ self.D_dot + self.I_r * self.K_alpha_dot + self.I_r_dot * self.K_alpha


In [81]:
dynamics = ModelDynamics()
dynamics.set_parameters(0, 4)
dynamics.test_function()

0.08706539182259396


In [70]:
dynamics = ModelDynamics()

dynamics.eval_M_dot(0, 2)

AttributeError: 'ModelDynamics' object has no attribute 'l_com'

R*(-R**2*x_A*sin(theta(t)) + R**2*y_A*cos(theta(t)) + x_A**3*sin(theta(t)) - x_A**2*y_A*cos(theta(t)) + x_A*y_A**2*sin(theta(t)) - y_A**3*cos(theta(t)))*Derivative(theta(t), t)/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)**2